In [1]:
import pandas as pd
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])
ques_info=ques_info.drop_duplicates('qid')
answer_info=pd.read_csv('answer_info.txt',sep='\s+',names=['aid','qid','uid','atime','ainfo','ainfoc','fine',
                                                                 'recommend','table','picture','video','numword',
                                                                 'thumbsup','thumbsdown','comment','collect','thank',
                                                                 'report','nohelp','oppose'])

In [2]:
atrain=train[train['target']==1]
atrain.columns=['qid','uid','atime','target']
answer_info=answer_info[['qid','uid','atime']]
answer_info=pd.concat([answer_info,atrain[['qid','uid','atime']]])


In [3]:
answer_info=answer_info.drop_duplicates(subset=['qid','uid'])

In [4]:
all_data=pd.concat([train[['qid','uid','time']],test])
def caul_time(x):
    time=x[1:].split('-H')
    return int(time[0])*24+int(time[1])
from tqdm import tqdm 
tqdm.pandas()
import numpy as np
answer_info['atime']=answer_info['atime'].progress_apply(lambda x:caul_time(x))
all_data['time']=all_data['time'].progress_apply(lambda x:caul_time(x))
all_data=pd.merge(all_data,answer_info,on='uid',how='inner')

100%|██████████| 10630880/10630880 [00:25<00:00, 417891.01it/s]


In [5]:
all_data['sub_time']=all_data['time']-all_data['atime']
all_data=all_data[all_data['sub_time']>0]
all_data=all_data[all_data['qid_x']!=all_data['qid_y']]

In [6]:
all_data['sort_num']=all_data.groupby(['qid_x','uid'])['sub_time'].rank(ascending=1,method='first')

In [7]:
all_data_1=all_data[all_data['sort_num']==1]

In [8]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
ALL=pd.concat([train[['qid','uid']],test[['qid','uid']]])

In [9]:
all_data_1=all_data_1[['qid_x','uid','qid_y']]
all_data_1.columns=['qid','uid','qid_1']

In [10]:
ALL=ALL.merge(all_data_1,on=['qid','uid'],how='left')

In [11]:
all_data_1=all_data[all_data['sort_num']==2]
all_data_1=all_data_1[['qid_x','uid','qid_y']]
all_data_1.columns=['qid','uid','qid_2']
ALL=ALL.merge(all_data_1,on=['qid','uid'],how='left')

In [12]:
all_data_1=all_data[all_data['sort_num']==3]
all_data_1=all_data_1[['qid_x','uid','qid_y']]
all_data_1.columns=['qid','uid','qid_3']
ALL=ALL.merge(all_data_1,on=['qid','uid'],how='left')

In [13]:
all_data_1=all_data[all_data['sort_num']==4]
all_data_1=all_data_1[['qid_x','uid','qid_y']]
all_data_1.columns=['qid','uid','qid_4']
ALL=ALL.merge(all_data_1,on=['qid','uid'],how='left')

In [14]:
all_data_1=all_data[all_data['sort_num']==5]
all_data_1=all_data_1[['qid_x','uid','qid_y']]
all_data_1.columns=['qid','uid','qid_5']
ALL=ALL.merge(all_data_1,on=['qid','uid'],how='left')

In [15]:
all_data_1=all_data[all_data['sort_num']==6]
all_data_1=all_data_1[['qid_x','uid','qid_y']]
all_data_1.columns=['qid','uid','qid_6']
ALL=ALL.merge(all_data_1,on=['qid','uid'],how='left')

In [16]:
ALL.to_hdf('last_answerlist_6.h5', key='data',index=False)

/root/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['qid', 'uid', 'qid_1', 'qid_2', 'qid_3', 'qid_4', 'qid_5', 'qid_6']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [17]:
all_data_1=all_data[all_data['sort_num']<=6]

In [18]:
all_data_1=all_data_1[['qid_x','uid','qid_y']]
all_data_1.columns=['qid_x','uid','qid']

In [19]:
ques_info=ques_info[['qid','qtopic']]
ques_info['qtopic']=ques_info['qtopic'].apply(lambda x:x.split(','))
all_data_1=all_data_1.merge(ques_info,on='qid',how='left')

In [20]:
all_data_1=all_data_1.groupby(['qid_x','uid'])['qtopic'].agg({'last_6_tpoic':'sum'})

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [21]:
all_data_1=all_data_1.reset_index()

In [22]:
all_data_1.columns=['qid', 'uid', 'last_6_tpoic']

In [23]:
all_data_1.to_hdf('last_answerlist_6_topic.h5', key='data',index=False)

/root/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['qid', 'uid', 'last_6_tpoic']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
